# Exercises - The Carry Trade

#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$

Use the data set `famabliss_strips_2025-11-28.xlsx`.

It gives prices on **zero coupon bonds** with maturities of 1 through 5 years.
* These are prices per \$1 face value on bonds that only pay principal.
* Such bonds can be created from treasuries by *stripping* out their coupons.
* In essence, you can consider these prices as the discount factors $Z$, for maturity intervals 1 through 5 years.

In this problem, we focus on six dates: the month of **November** in 2020 through 2025.

In [2]:
import numpy as np
import pandas as pd

# Load the data
df = pd.read_excel("/Users/lin_/Desktop/37400 FixedIncome/data/famabliss_strips_2025-11-28.xlsx")
df['date'] = pd.to_datetime(df['date'])

# Filter for November dates from 2020-2025
nov_data = df[(df['date'].dt.month == 11) & (df['date'].dt.year.isin([2020, 2021, 2022, 2023, 2024, 2025]))].copy()
nov_data = nov_data.set_index(nov_data['date'].dt.year)
nov_data = nov_data.drop(columns=['date'])
nov_data.columns = [f'Z_{i}yr' for i in range(1, 6)]
nov_data.index.name = 'Year'
print("Zero-Coupon Bond Prices (Discount Factors) - November Data")
print(nov_data.round(6))

Zero-Coupon Bond Prices (Discount Factors) - November Data
         Z_1yr     Z_2yr     Z_3yr     Z_4yr     Z_5yr
Year                                                  
2020  0.998839  0.997047  0.994464  0.988809  0.981515
2021  0.997523  0.988614  0.974796  0.958322  0.943772
2022  0.954375  0.918220  0.887608  0.857356  0.830593
2023  0.951080  0.911951  0.876957  0.842191  0.809378
2024  0.959187  0.920923  0.885440  0.850418  0.817795
2025  0.964936  0.932866  0.900919  0.868240  0.836057


# The Carry Trade

## 1.1

Suppose it is `November 2020`, and you determine to implement a carry trade with the following specification...

* Long `$100` million (market value, not face value) of the 5-year zero-coupon bond (maturing `Nov 2025`.)
* Short `$100` million (market value, not face value) of the 1-year zero-coupon bond (maturing `Nov 2021`.)
* Assume there is a `2%` haircut on each side of the trade, so it requires `$4` million of investor capital to initiate it.

1. Calculate the total profit and loss year-by-year.
1. Calculate the total return (`Nov 2025`) on the initial \\$4 million of investor capital.

#### Short position
* Each year you will roll over the short position to maintain a short `$100` million (market value) in the 1-year bond.
* This will require injecting more cash into the trade, as the expiring short will require more than `$100` million to close out. 
* In `Nov 2024`, no need to open a new short position, as your long position will (at that point) be a one-year bond.

#### Alternatives
The scheme above is for simplicity. You could try more interesting ways of setting the short position...
* Open a new short position sized to whatever is needed to cover the expiring short position.
* Set the short positions to duration-hedge the long position.

### Solution 1.1

In [3]:
# Extract the discount factors
Z = nov_data.values
years = nov_data.index.values

# Initial parameters
long_mv_initial = 100_000_000  # $100 million market value
short_mv_target = 100_000_000  # $100 million market value
haircut = 0.02  # 2% on each side
initial_capital = long_mv_initial * haircut + short_mv_target * haircut  # $4 million

print(f"Initial investor capital required: ${initial_capital:,.0f}")
print(f"Haircut per side: {haircut:.0%}")

Initial investor capital required: $4,000,000
Haircut per side: 2%


In [4]:
# November 2020 prices
Z_5yr_2020 = nov_data.loc[2020, 'Z_5yr']  # 5-year bond price in Nov 2020
Z_1yr_2020 = nov_data.loc[2020, 'Z_1yr']  # 1-year bond price in Nov 2020

# LONG POSITION: $100M market value of 5-year bond
# Face value = Market value / Price
long_face_value = long_mv_initial / Z_5yr_2020
print(f"Nov 2020 - LONG POSITION SETUP:")
print(f"  5-year zero price (Z_5): {Z_5yr_2020:.6f}")
print(f"  Long market value: ${long_mv_initial:,.0f}")
print(f"  Long face value: ${long_face_value:,.2f}")

# SHORT POSITION: $100M market value of 1-year bond
short_face_value_2020 = short_mv_target / Z_1yr_2020
print(f"\n  1-year zero price (Z_1): {Z_1yr_2020:.6f}")
print(f"  Short market value: ${short_mv_target:,.0f}")
print(f"  Short face value: ${short_face_value_2020:,.2f}")

Nov 2020 - LONG POSITION SETUP:
  5-year zero price (Z_5): 0.981515
  Long market value: $100,000,000
  Long face value: $101,883,281.16

  1-year zero price (Z_1): 0.998839
  Short market value: $100,000,000
  Short face value: $100,116,227.51


In [5]:
# Track positions and P&L year by year
results = []

# Store the long face value (fixed throughout)
long_fv = long_face_value

# Track short positions
short_fv = short_face_value_2020  # Initial short face value
cumulative_cash_injection = 0

print("="*80)
print("YEAR-BY-YEAR CARRY TRADE ANALYSIS")
print("="*80)

for i, year in enumerate([2020, 2021, 2022, 2023, 2024, 2025]):
    remaining_years = 5 - i  # Years to maturity for long position
    
    if remaining_years > 0:
        long_price = nov_data.loc[year, f'Z_{remaining_years}yr']
    else:
        long_price = 1.0  # At maturity
    
    long_mv = long_fv * long_price
    
    print(f"\n{'='*80}")
    print(f"NOVEMBER {year}")
    print(f"{'='*80}")
    
    print(f"\nLONG POSITION:")
    print(f"  Remaining maturity: {remaining_years} year(s)")
    print(f"  Bond price: {long_price:.6f}")
    print(f"  Face value: ${long_fv:,.2f}")
    print(f"  Market value: ${long_mv:,.2f}")
    
    if year == 2020:
        # Initial setup
        short_price = nov_data.loc[year, 'Z_1yr']
        short_mv = short_fv * short_price
        pnl_long = 0
        pnl_short = 0
        cash_injection = 0
        
        print(f"\nSHORT POSITION:")
        print(f"  1-year bond price: {short_price:.6f}")
        print(f"  Face value shorted: ${short_fv:,.2f}")
        print(f"  Market value: ${short_mv:,.2f}")
        print(f"\n  [Initial setup - no P&L yet]")
        
    elif year == 2025:
        # Final settlement - long matures at face value
        long_mv = long_fv  # Receive face value
        
        # Get previous year's long value
        prev_long_price = nov_data.loc[2024, 'Z_1yr']
        prev_long_mv = long_fv * prev_long_price
        pnl_long = long_mv - prev_long_mv
        
        # No short position in final year
        pnl_short = 0
        cash_injection = 0
        
        print(f"\n  [Bond matures - receive face value]")
        print(f"  Previous MV (Nov 2024): ${prev_long_mv:,.2f}")
        print(f"  Maturity value: ${long_mv:,.2f}")
        print(f"  Long P&L: ${pnl_long:,.2f}")
        print(f"\nSHORT POSITION:")
        print(f"  [No short position - closed in Nov 2024]")
        
    else:
        # Years 2021-2024: Short expires, roll over
        
        # Close expiring short: pay face value
        short_close_cost = short_fv  # Pay face value at maturity
        
        # Short P&L: received MV when shorted, pay face value to close
        pnl_short = short_mv_target - short_close_cost  # This is negative (loss)
        
        # Calculate long P&L
        prev_remaining = 5 - (i - 1)
        prev_long_price = nov_data.loc[years[i-1], f'Z_{prev_remaining}yr']
        prev_long_mv = long_fv * prev_long_price
        pnl_long = long_mv - prev_long_mv
        
        print(f"\n  Previous year MV: ${prev_long_mv:,.2f}")
        print(f"  Long P&L: ${pnl_long:,.2f}")
        
        print(f"\nSHORT POSITION ROLLOVER:")
        print(f"  Closing expiring short:")
        print(f"    Original short face value: ${short_fv:,.2f}")
        print(f"    Cost to close (pay face value): ${short_close_cost:,.2f}")
        print(f"    Originally received (MV): ${short_mv_target:,.2f}")
        print(f"    Short P&L: ${pnl_short:,.2f}")
        
        if year < 2024:
            # Open new short position at $100M MV
            new_short_price = nov_data.loc[year, 'Z_1yr']
            new_short_fv = short_mv_target / new_short_price
            
            # Cash injection needed: close old short - open new short
            cash_injection = short_close_cost - short_mv_target
            
            print(f"\n  Opening new short:")
            print(f"    New 1-year price: {new_short_price:.6f}")
            print(f"    New short face value: ${new_short_fv:,.2f}")
            print(f"    New short MV: ${short_mv_target:,.2f}")
            print(f"\n  Cash injection needed: ${cash_injection:,.2f}")
            
            # Update short position for next year
            short_fv = new_short_fv
        else:
            # Nov 2024: Close short, don't open new one (long is now 1-year)
            cash_injection = short_close_cost - short_mv_target
            print(f"\n  [Not opening new short - long position is now 1-year]")
            print(f"  Cash to close: ${cash_injection:,.2f}")
    
    total_pnl = pnl_long + pnl_short
    if year > 2020:
        cumulative_cash_injection += cash_injection if year <= 2024 else 0
    
    print(f"\n  YEAR {year} TOTAL P&L: ${total_pnl:,.2f}")
    
    results.append({
        'Year': year,
        'Long_MV': long_mv,
        'Long_PnL': pnl_long,
        'Short_PnL': pnl_short,
        'Total_PnL': total_pnl,
        'Cash_Injection': cash_injection if year > 2020 and year <= 2024 else 0
    })

results_df = pd.DataFrame(results)

YEAR-BY-YEAR CARRY TRADE ANALYSIS

NOVEMBER 2020

LONG POSITION:
  Remaining maturity: 5 year(s)
  Bond price: 0.981515
  Face value: $101,883,281.16
  Market value: $100,000,000.00

SHORT POSITION:
  1-year bond price: 0.998839
  Face value shorted: $100,116,227.51
  Market value: $100,000,000.00

  [Initial setup - no P&L yet]

  YEAR 2020 TOTAL P&L: $0.00

NOVEMBER 2021

LONG POSITION:
  Remaining maturity: 4 year(s)
  Bond price: 0.958322
  Face value: $101,883,281.16
  Market value: $97,637,027.12

  Previous year MV: $100,000,000.00
  Long P&L: $-2,362,972.88

SHORT POSITION ROLLOVER:
  Closing expiring short:
    Original short face value: $100,116,227.51
    Cost to close (pay face value): $100,116,227.51
    Originally received (MV): $100,000,000.00
    Short P&L: $-116,227.51

  Opening new short:
    New 1-year price: 0.997523
    New short face value: $100,248,347.30
    New short MV: $100,000,000.00

  Cash injection needed: $116,227.51

  YEAR 2021 TOTAL P&L: $-2,479,200.

In [6]:
# Summary table
print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)

summary_df = results_df.copy()
summary_df['Cumulative_PnL'] = summary_df['Total_PnL'].cumsum()
summary_df['Cumulative_Cash_Injection'] = summary_df['Cash_Injection'].cumsum()

# Format for display
for col in ['Long_MV', 'Long_PnL', 'Short_PnL', 'Total_PnL', 'Cash_Injection', 'Cumulative_PnL', 'Cumulative_Cash_Injection']:
    summary_df[col] = summary_df[col].apply(lambda x: f"${x:,.0f}")

print(summary_df.to_string(index=False))


SUMMARY TABLE
 Year      Long_MV    Long_PnL   Short_PnL   Total_PnL Cash_Injection Cumulative_PnL Cumulative_Cash_Injection
 2020 $100,000,000          $0          $0          $0             $0             $0                        $0
 2021  $97,637,027 $-2,362,973   $-116,228 $-2,479,200       $116,228    $-2,479,200                  $116,228
 2022  $90,432,372 $-7,204,655   $-248,347 $-7,453,003       $248,347    $-9,932,203                  $364,575
 2023  $92,912,546  $2,480,175 $-4,780,616 $-2,300,442     $4,780,616   $-12,232,644                $5,145,191
 2024  $97,725,113  $4,812,566 $-5,143,642   $-331,076     $5,143,642   $-12,563,720               $10,288,833
 2025 $101,883,281  $4,158,168          $0  $4,158,168             $0    $-8,405,552               $10,288,833


In [7]:
# Calculate total return
total_pnl = results_df['Total_PnL'].sum()
total_cash_injections = results_df['Cash_Injection'].sum()

# Total capital invested = initial capital + all cash injections
total_capital_invested = initial_capital + total_cash_injections

# Total return on initial capital
total_return_initial = total_pnl / initial_capital

# Total return on all capital invested
total_return_all_capital = total_pnl / total_capital_invested

print("\n" + "="*80)
print("TOTAL RETURN CALCULATION")
print("="*80)
print(f"\nInitial capital: ${initial_capital:,.0f}")
print(f"Total cash injections: ${total_cash_injections:,.0f}")
print(f"Total capital invested: ${total_capital_invested:,.0f}")
print(f"\nTotal P&L (Nov 2020 - Nov 2025): ${total_pnl:,.0f}")
print(f"\nTotal return on initial $4M capital: {total_return_initial:.2%}")
print(f"Total return on all capital invested: {total_return_all_capital:.2%}")


TOTAL RETURN CALCULATION

Initial capital: $4,000,000
Total cash injections: $10,288,833
Total capital invested: $14,288,833

Total P&L (Nov 2020 - Nov 2025): $-8,405,552

Total return on initial $4M capital: -210.14%
Total return on all capital invested: -58.83%


## 1.2

How would this trade play out if the path of one-year spot rates equaled the forward rates observed in `2020`?

### Solution 1.2

Under the Expectations Hypothesis (EH), forward rates are unbiased predictors of future spot rates. If the actual spot rates followed the forward rates implied in November 2020, the trade would break even (zero profit) because:

1. The forward rates embedded in the yield curve reflect the market's risk-neutral expectation of future rates
2. If these expectations were realized, holding a long-term bond would earn the same return as rolling over short-term bonds



In [8]:
# Calculate implied forward rates from Nov 2020 discount factors
# Forward rate f(t,t+1) from year t to t+1: (1+f) = Z_t / Z_{t+1}

print("FORWARD RATES IMPLIED BY NOV 2020 YIELD CURVE")
print("="*60)

Z_2020 = nov_data.loc[2020].values  # [Z_1, Z_2, Z_3, Z_4, Z_5]

# Calculate 1-year forward rates
# f(0,1) is the 1-year spot rate
# f(1,2) is the 1-year forward rate starting in year 1
# etc.

forward_rates = []
forward_prices = []

# f(0,1): 1-year spot rate
f_0_1 = 1/Z_2020[0] - 1
forward_rates.append(f_0_1)
forward_prices.append(Z_2020[0])
print(f"f(0,1) - Nov 2020 to Nov 2021: {f_0_1:.4%}")
print(f"  Implied 1-yr zero price: {Z_2020[0]:.6f}")

# f(1,2): Forward rate from year 1 to year 2
f_1_2 = Z_2020[0]/Z_2020[1] - 1
forward_rates.append(f_1_2)
forward_prices.append(Z_2020[1]/Z_2020[0])
print(f"f(1,2) - Nov 2021 to Nov 2022: {f_1_2:.4%}")
print(f"  Implied 1-yr zero price: {Z_2020[1]/Z_2020[0]:.6f}")

# f(2,3): Forward rate from year 2 to year 3
f_2_3 = Z_2020[1]/Z_2020[2] - 1
forward_rates.append(f_2_3)
forward_prices.append(Z_2020[2]/Z_2020[1])
print(f"f(2,3) - Nov 2022 to Nov 2023: {f_2_3:.4%}")
print(f"  Implied 1-yr zero price: {Z_2020[2]/Z_2020[1]:.6f}")

# f(3,4): Forward rate from year 3 to year 4
f_3_4 = Z_2020[2]/Z_2020[3] - 1
forward_rates.append(f_3_4)
forward_prices.append(Z_2020[3]/Z_2020[2])
print(f"f(3,4) - Nov 2023 to Nov 2024: {f_3_4:.4%}")
print(f"  Implied 1-yr zero price: {Z_2020[3]/Z_2020[2]:.6f}")

# f(4,5): Forward rate from year 4 to year 5
f_4_5 = Z_2020[3]/Z_2020[4] - 1
forward_rates.append(f_4_5)
forward_prices.append(Z_2020[4]/Z_2020[3])
print(f"f(4,5) - Nov 2024 to Nov 2025: {f_4_5:.4%}")
print(f"  Implied 1-yr zero price: {Z_2020[4]/Z_2020[3]:.6f}")

FORWARD RATES IMPLIED BY NOV 2020 YIELD CURVE
f(0,1) - Nov 2020 to Nov 2021: 0.1162%
  Implied 1-yr zero price: 0.998839
f(1,2) - Nov 2021 to Nov 2022: 0.1797%
  Implied 1-yr zero price: 0.998206
f(2,3) - Nov 2022 to Nov 2023: 0.2597%
  Implied 1-yr zero price: 0.997409
f(3,4) - Nov 2023 to Nov 2024: 0.5719%
  Implied 1-yr zero price: 0.994314
f(4,5) - Nov 2024 to Nov 2025: 0.7431%
  Implied 1-yr zero price: 0.992624


In [9]:
# Compare actual 1-year spot rates to forward rates
print("\nCOMPARISON: ACTUAL vs IMPLIED FORWARD RATES")
print("="*60)

actual_rates = []
for year in [2020, 2021, 2022, 2023, 2024]:
    Z_1 = nov_data.loc[year, 'Z_1yr']
    r = 1/Z_1 - 1
    actual_rates.append(r)

comparison_df = pd.DataFrame({
    'Period': ['Nov 2020-2021', 'Nov 2021-2022', 'Nov 2022-2023', 'Nov 2023-2024', 'Nov 2024-2025'],
    'Forward Rate (2020)': forward_rates,
    'Actual Spot Rate': actual_rates,
    'Difference': [a - f for a, f in zip(actual_rates, forward_rates)]
})

for col in ['Forward Rate (2020)', 'Actual Spot Rate', 'Difference']:
    comparison_df[col] = comparison_df[col].apply(lambda x: f"{x:.4%}")

print(comparison_df.to_string(index=False))


COMPARISON: ACTUAL vs IMPLIED FORWARD RATES
       Period Forward Rate (2020) Actual Spot Rate Difference
Nov 2020-2021             0.1162%          0.1162%    0.0000%
Nov 2021-2022             0.1797%          0.2483%    0.0686%
Nov 2022-2023             0.2597%          4.7806%    4.5209%
Nov 2023-2024             0.5719%          5.1436%    4.5718%
Nov 2024-2025             0.7431%          4.2550%    3.5119%


In [10]:
# Calculate the trade P&L if forward rates had been realized
print("\nHYPOTHETICAL TRADE IF FORWARD RATES WERE REALIZED")
print("="*60)

# Under EH, if forward rates are realized:
# - The long 5-year bond value would evolve based on forward prices
# - The short 1-year bond P&L would exactly offset any carry

# Initial long position: $100M / Z_5 = face value
Z_5_2020 = Z_2020[4]
long_fv_hyp = long_mv_initial / Z_5_2020

# Under forward rate scenario, the n-year bond price at time t
# equals the product of forward prices from t to n

# Calculate hypothetical prices for the long bond over time
# In Nov 2021, the 5-year bond becomes a 4-year bond
# Its price should be Z_4/Z_1 (if forwards realized) = Z_2020[3]/Z_2020[0]

print(f"\nIf forward rates were realized:")
print(f"  All risk premia would be zero")
print(f"  Long bond return = Short bond return (compounded)")
print(f"  Carry trade P&L ≈ 0")

# The theoretical result under pure EH
long_terminal = long_fv_hyp  # Receive face value
short_cost = long_mv_initial / Z_5_2020  # Cost of rolling shorts = face value of long

print(f"\n  Long terminal value: ${long_terminal:,.2f}")
print(f"  Cost of rolling shorts (if EH holds): ${short_cost:,.2f}")
print(f"  Net P&L (theoretical): $0")

# Show why this is true mathematically
print(f"\nMathematical proof:")
print(f"  Long return = (1/Z_5) = {1/Z_5_2020:.6f}")
print(f"  Rolling short return = (1/Z_1)*(Z_1/Z_2)*(Z_2/Z_3)*(Z_3/Z_4)*(Z_4/Z_5)")
print(f"                       = 1/Z_5 = {1/Z_5_2020:.6f}")
print(f"  Net profit = 0 under EH")


HYPOTHETICAL TRADE IF FORWARD RATES WERE REALIZED

If forward rates were realized:
  All risk premia would be zero
  Long bond return = Short bond return (compounded)
  Carry trade P&L ≈ 0

  Long terminal value: $101,883,281.16
  Cost of rolling shorts (if EH holds): $101,883,281.16
  Net P&L (theoretical): $0

Mathematical proof:
  Long return = (1/Z_5) = 1.018833
  Rolling short return = (1/Z_1)*(Z_1/Z_2)*(Z_2/Z_3)*(Z_3/Z_4)*(Z_4/Z_5)
                       = 1/Z_5 = 1.018833
  Net profit = 0 under EH


## 1.3

Given Fact 3 of the *dynamic* (conditional) tests of the Expectations Hypothesis (EH), do you expect that as of `Nov 2025` the long-short trade above looks more or less favorable for `Nov 2025-2030` than it did for `Nov 2020-2025`?

### Solution 1.3

**Fact 3 of the dynamic tests of the Expectations Hypothesis** states that:
> The slope of the yield curve predicts bond excess returns. A steeper curve predicts higher returns to long bonds relative to short bonds.

This means carry trades tend to be more profitable when the yield curve is steep (positively sloped) and less profitable when it's flat or inverted.

Let's compare the yield curve slopes:

In [11]:
# Calculate spot rates and yield curve slope for 2020 vs 2025
print("YIELD CURVE COMPARISON: NOV 2020 vs NOV 2025")
print("="*60)

# Spot rates: r_n = (1/Z_n)^(1/n) - 1 (annually compounded)
# Or simply: r_n = -ln(Z_n)/n (continuously compounded)

def calc_spot_rates(row):
    rates = []
    for i, col in enumerate(row.index):
        n = i + 1
        Z = row[col]
        r = (1/Z)**(1/n) - 1  # Annually compounded
        rates.append(r)
    return rates

rates_2020 = calc_spot_rates(nov_data.loc[2020])
rates_2025 = calc_spot_rates(nov_data.loc[2025])

print("\nSpot Rates (annually compounded):")
print(f"{'Maturity':<12} {'Nov 2020':>12} {'Nov 2025':>12}")
print("-"*36)
for i, (r20, r25) in enumerate(zip(rates_2020, rates_2025)):
    print(f"{i+1}-year{'':<6} {r20:>11.4%} {r25:>11.4%}")

# Yield curve slope: 5-year rate minus 1-year rate
slope_2020 = rates_2020[4] - rates_2020[0]
slope_2025 = rates_2025[4] - rates_2025[0]

print(f"\nYield Curve Slope (5yr - 1yr):")
print(f"  Nov 2020: {slope_2020:.4%}")
print(f"  Nov 2025: {slope_2025:.4%}")

YIELD CURVE COMPARISON: NOV 2020 vs NOV 2025

Spot Rates (annually compounded):
Maturity         Nov 2020     Nov 2025
------------------------------------
1-year           0.1162%     3.6339%
2-year           0.1480%     3.5358%
3-year           0.1852%     3.5392%
4-year           0.2817%     3.5953%
5-year           0.3739%     3.6461%

Yield Curve Slope (5yr - 1yr):
  Nov 2020: 0.2576%
  Nov 2025: 0.0122%


In [12]:
print("\nINTERPRETATION")
print("="*60)

if slope_2025 > slope_2020:
    comparison = "steeper"
    expectation = "more favorable"
else:
    comparison = "flatter (or less steep)"
    expectation = "less favorable"

print(f"\nThe yield curve in Nov 2025 is {comparison} than in Nov 2020.")
print(f"\nBased on Fact 3 of the dynamic EH tests:")
print(f"  - A steeper curve predicts higher excess returns to long bonds")
print(f"  - The carry trade for Nov 2025-2030 looks {expectation}")
print(f"    compared to Nov 2020-2025")

# Additional context
print(f"\nKey insight:")
print(f"  Nov 2020 had near-zero short rates ({rates_2020[0]:.4%}) with modest 5yr rates ({rates_2020[4]:.4%})")
print(f"  Nov 2025 has higher short rates ({rates_2025[0]:.4%}) but similar spread")
print(f"")
print(f"  The term premium (excess return for bearing duration risk) tends to be")
print(f"  larger when the curve is steep, making carry trades more attractive.")


INTERPRETATION

The yield curve in Nov 2025 is flatter (or less steep) than in Nov 2020.

Based on Fact 3 of the dynamic EH tests:
  - A steeper curve predicts higher excess returns to long bonds
  - The carry trade for Nov 2025-2030 looks less favorable
    compared to Nov 2020-2025

Key insight:
  Nov 2020 had near-zero short rates (0.1162%) with modest 5yr rates (0.3739%)
  Nov 2025 has higher short rates (3.6339%) but similar spread

  The term premium (excess return for bearing duration risk) tends to be
  larger when the curve is steep, making carry trades more attractive.


***